In [1]:
import pandas as pd
import numpy as np
import pickle

# enter your path to the datasets here!
tl_path = 'data/timeseries.csv'
stays_path = 'data/stays.csv'

from preprocess import load_dataframes, generate_data_dict
from traintest import train_test_split

## USE THESE TO GENERATE DATA DICT FROM SCRATCH - Takes about 2 hours
# tl, stays = load_dataframes(tl_path, stays_path)
# data_dict = generate_data_dict(tl)


## USE THIS TO LOAD DATA DICT FROM PKL FILE
with open('data/data_dict.pkl', 'rb') as file:
    data_dict = pickle.load(file)

X_train, X_test, y_train, y_test = train_test_split(data_dict)

In [ ]:
from pygam import LogisticGAM

log_gam = LogisticGAM(n_splines=10).gridsearch(X_train, y_train)

  0% (0 of 11) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--

In [ ]:
log_gam.accuracy(X_test, y_test)